# Online linear learning

In [1]:
from abc import ABCMeta, abstractmethod

In [2]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [6]:
class BaseLinearOnline(BaseEstimator, metaclass=ABCMeta):
    @abstractmethod
    def __init__(self, average, n_iter, random_state, shuffle):
        self.average      = average
        self.n_iter       = n_iter
        self.random_state = random_state
        self.shuffle      = shuffle

    def fit(self, X, y):
        X, y                      = check_X_y(X, y)
        self.label_binarizer_     = LabelBinarizer(neg_label=-1, pos_label=1)
        Y                         = self.label_binarizer_.fit_transform(y)
        n_samples, n_features     = X.shape
        n_samples, n_classes      = Y.shape
        self.coef_                = np.zeros((n_features, n_classes))

        if self.average:
            average_coef          = np.zeros((n_features, n_classes))

        t                         = 1

        for epoch in range(self.n_iter):
            if self.shuffle:
                X, Y              = shuffle(X, Y, random_state=self.random_state)

            for i in range(n_samples):
                self._update(X[i:i + 1], Y[i:i + 1])

                if self.average:
                    average_coef *= t / (t + 1.0)
                    average_coef += 1.0 / (t + 1.0) * self.coef_

                t                 = t + 1

        if self.average:
            self.coef_            = average_coef

        return self

    @abstractmethod
    def _update(self, X, Y):
        pass

In [4]:
class LinearClassifierMixin(ClassifierMixin):
    def decision_function(self, X):
        return X @ self.coef_

    def predict(self, X):
        check_is_fitted(self, ['coef_'])

        X = check_array(X)
        
        return self.label_binarizer_.inverse_transform(self.decision_function(X))

## Perceptron

\begin{align}
    \mathbf{w}_{t + 1} = \begin{cases}
        \mathbf{w}_{t} + \eta y_{t} \mathbf{x}_{t} & \text{if} \quad y_{t} \langle \mathbf{w}_{t}, \mathbf{x}_{t} \rangle \leq 0 \\
        \mathbf{w}_{t}                             & \text{otherwise}
    \end{cases}
\end{align}

In [5]:
class Perceptron(BaseLinearOnline, LinearClassifierMixin):
    def __init__(
        self,              average=False, lr=1.0, n_iter=5,
        random_state=None, shuffle=True
    ):
        super(Perceptron, self).__init__(
            average=average,           n_iter=n_iter,
            random_state=random_state, shuffle=shuffle
        )

        self.lr = lr

    def _update(self, X, Y):
        P           = self.decision_function(X)
        self.coef_ += self.lr * np.where(Y * P <= 0.0, X.T @ Y, 0.0)

## Perceptron with uneven margins [2]

\begin{align}
    \mathbf{w}_{t + 1} = \begin{cases}
        \mathbf{w}_{t} + \eta y_{t} \mathbf{x}_{t} & \text{if} \quad y_{t} \langle \mathbf{w}_{t}, \mathbf{x}_{t} \rangle \leq \tau_{y_{t}} \\
        \mathbf{w}_{t}                             & \text{otherwise}
    \end{cases}
\end{align}

where $\tau_{-1}, \tau_{+1}$ are fixed margin parameters chosen before learning.

In [6]:
class PAUM(BaseLinearOnline, LinearClassifierMixin):
    def __init__(
        self,         lr=1.0,       n_iter=5,    random_state=None,
        shuffle=True, tau_neg=-1.0, tau_pos=1.0
    ):
        super(PAUM, self).__init__(
            average=False,           n_iter=n_iter,
            random_state=random_state, shuffle=shuffle
        )

        self.lr      = lr
        self.tau_neg = tau_neg
        self.tau_pos = tau_pos

    def _update(self, X, Y):
        margin_params = np.where(Y == -1, self.tau_neg, self.tau_pos)
        P             = self.decision_function(X)
        self.coef_   += self.lr * np.where(Y * P <= margin_params, X.T @ Y, 0.0)

## Passive aggressive [1]

\begin{align}
    \mathbf{w}_{t + 1} = \mathbf{w}_{t} + \eta y_{t} \mathbf{x}_{t}
\end{align}

- PA

\begin{align}
    \mathbf{w}_{t + 1} &= \underset{\mathbf{w} \in \mathbb{R}^{d}}{\mathrm{argmin}} \frac{1}{2} \| \mathbf{w} - \mathbf{w}_{t} \|^{2} + \infty H(\mathbf{x}_{t}, y_{t}, \mathbf{w}) \\
    \eta               &= \frac{H(\mathbf{x}_{t}, y_{t}, \mathbf{w}_{t})}{\| \mathbf{x}_{t} \|^{2}}
\end{align}

- PA-I

\begin{align}
    \mathbf{w}_{t + 1} &= \underset{\mathbf{w} \in \mathbb{R}^{d}}{\mathrm{argmin}} \frac{1}{2} \| \mathbf{w} - \mathbf{w}_{t} \|^{2} + C H(\mathbf{x}_{t}, y_{t}, \mathbf{w}) \\
    \eta               &= \min (\frac{H(\mathbf{x}_{t}, y_{t}, \mathbf{w}_{t})}{\| \mathbf{x}_{t} \|^{2}}, C)
\end{align}

- PA-II

\begin{align}
    \mathbf{w}_{t + 1} &= \underset{\mathbf{w} \in \mathbb{R}^{d}}{\mathrm{argmin}} \frac{1}{2} \| \mathbf{w} - \mathbf{w}_{t} \|^{2} + C H(\mathbf{x}_{t}, y_{t}, \mathbf{w})^{2} \\
    \eta               &= \frac{H(\mathbf{x}_{t}, y_{t}, \mathbf{w}_{t})}{\| \mathbf{x}_{t} \|^{2} + \frac{1}{2 C}}
\end{align}

In [7]:
class PassiveAggressiveClassifier(BaseLinearOnline, LinearClassifierMixin):
    def __init__(
        self,              C=1.0,        loss='hinge', n_iter=5,
        random_state=None, shuffle=True
    ):
        super(PassiveAggressiveClassifier, self).__init__(
            average=False,             n_iter=n_iter,
            random_state=random_state, shuffle=shuffle
        )

        self.C    = C
        self.loss = loss

    def _update(self, X, Y):
        P           = self.decision_function(X)
        hinge_loss  = np.maximum(1.0 - Y * P, 0.0)

        if self.loss == 'hinge':
            eta     = np.minimum(hinge_loss, self.C) / (X @ X.T)
        elif self.loss == 'squared_hinge':
            eta     = hinge_loss / (X @ X.T + 0.5 / self.C)

        self.coef_ += eta * (X.T @ Y)

## Example: usps

In [ ]:
usps           = fetch_mldata('usps')
X              = usps.data
y              = usps.target
X_train        = X[:7291]
X_test         = X[7291:]
y_train        = y[:7291]
y_test         = y[7291:]

score          = GridSearchCV(
    estimator  = Perceptron(random_state=0),
    param_grid = {'lr': (0.001, 0.01, 0.1, 1.0)}
).fit(X_train, y_train).score(X_test, y_test)
print('Perceptron:          {0:.4f}'.format(score))

score          = GridSearchCV(
    estimator  = Perceptron(average=True, random_state=0),
    param_grid = {'lr': (0.001, 0.01, 0.1, 1.0)}
).fit(X_train, y_train).score(X_test, y_test)
print('Averaged Perceptron: {0:.4f}'.format(score))

score          = GridSearchCV(
    estimator  = PAUM(random_state=0),
    param_grid = {'lr': (0.001, 0.01, 0.1, 1.0)}
).fit(X_train, y_train).score(X_test, y_test)
print('PAUM(-1, 1):         {0:.4f}'.format(score))

score          = GridSearchCV(
    estimator  = PassiveAggressiveClassifier(random_state=0),
    param_grid = {'C': (0.001, 0.01, 0.1, 1.0)}
).fit(X_train, y_train).score(X_test, y_test)
print('PA-I:                {0:.4f}'.format(score))

score          = GridSearchCV(
    estimator  = PassiveAggressiveClassifier(loss='squared_hinge', random_state=0),
    param_grid = {'C': (0.001, 0.01, 0.1, 1.0)}
).fit(X_train, y_train).score(X_test, y_test)
print('PA-II:               {0:.4f}'.format(score))

## TODO

The following implementation:

- multiclass PA [Crammer+, JMLR, 2006]
- CW [Dredze+, ICML, 2008]
- AROW [Crammer+, NIPS, 2009]
- SCW [Wang+, ICML, 2012]

## References

1. K. Crammer, O. Dekel, J. Keshet , S. Shalev-Shwartz and Y. Singer, "[Online Passive-Aggressive Algorithms](http://www.jmlr.org/papers/volume7/crammer06a/crammer06a.pdf)," JMLR, 2006.
2. Y. Li, H. Zaragoza, R. Herbrich, J. Shawe-Taylor, and J. Kandola, "[The Perceptron Algorithm with Uneven Margins](https://pdfs.semanticscholar.org/2285/2d7c76d517adb6f5ebe4f70c0b3c3e9ee24c.pdf)," in NIPS, 2002.